In [23]:
import os
from pathlib import Path

from itertools import chain, product

import math
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

import time
from datetime import datetime, timedelta
import pymmwr

from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb

# https://github.com/reichlab/timeseriesutils
from timeseriesutils import featurize

In [24]:
# config settings

# date of forecast generation
forecast_date = datetime.today().date()
#forecast_date = datetime(2025, 2, 11).date()
print(forecast_date)
 #"2024-10-13", "2024-11-24", "2024-12-22"

# next Saturday: weekly forecasts are relative to this date
ref_date = forecast_date - timedelta((forecast_date.weekday() + 2) % 7 - 7)
print(f'reference date = {ref_date}')

# maximum forecast horizon
max_horizon = 5

# bagging setup
num_bags = 100
bag_frac_samples = 0.7

# quantile levels at which to generate predictions
q_levels = [0.025, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.975]
q_labels = ['0.025', '0.05', '0.1', '0.25', '0.5', '0.75',  '0.9', '0.95', '0.975']


2025-02-25
reference date = 2025-03-01


In [25]:
os.chdir('/Users/dk29776/Dropbox/UTAustin/City-Level-Forecasting')

In [26]:
import sys
sys.path.append('/Users/dk29776/Dropbox/UTAustin/City-Level-Forecasting/GBM/code')
import loader 


In [27]:
dat = pd.read_csv('https://raw.githubusercontent.com/reichlab/flu-metrocast/main/target-data/oracle-output.csv')
dat = dat[dat["target"] == "ILI ED visits"]
dat = dat[dat["location"] != "Unknown"]
dat.target_end_date.max()
dat

,target_end_date,location,target,oracle_value
0,2016-01-02,Bronx,ILI ED visits,165.0
1,2016-01-02,Brooklyn,ILI ED visits,166.0
2,2016-01-02,Manhattan,ILI ED visits,84.0
3,2016-01-02,NYC,ILI ED visits,744.0
4,2016-01-02,Queens,ILI ED visits,280.0
...,...,...,...,...
3488,2025-02-22,Brooklyn,ILI ED visits,1051.0
3489,2025-02-22,Manhattan,ILI ED visits,486.0
3490,2025-02-22,NYC,ILI ED visits,3621.0
3491,2025-02-22,Queens,ILI ED visits,954.0


In [28]:
nyc_population = pd.read_csv('/Users/dk29776/Dropbox/UTAustin/flu-metrocast/auxiliary-data/nyc_population.csv')
nyc_population

,location,population
0,Bronx,1356476
1,Brooklyn,2561225
2,Manhattan,1597451
3,Queens,2252196
4,Staten Island,490687
5,NYC,8258035


In [29]:
dat = dat.merge(nyc_population, on='location', how='left')
dat

,target_end_date,location,target,oracle_value,population
0,2016-01-02,Bronx,ILI ED visits,165.0,1356476
1,2016-01-02,Brooklyn,ILI ED visits,166.0,2561225
2,2016-01-02,Manhattan,ILI ED visits,84.0,1597451
3,2016-01-02,NYC,ILI ED visits,744.0,8258035
4,2016-01-02,Queens,ILI ED visits,280.0,2252196
...,...,...,...,...,...
2863,2025-02-22,Brooklyn,ILI ED visits,1051.0,2561225
2864,2025-02-22,Manhattan,ILI ED visits,486.0,1597451
2865,2025-02-22,NYC,ILI ED visits,3621.0,8258035
2866,2025-02-22,Queens,ILI ED visits,954.0,2252196


In [30]:
dat['inc'] = dat['oracle_value']/(dat['population']/ 100000)
dat['target_end_date'] = pd.to_datetime(dat['target_end_date'])
dat.rename(columns={'population': 'pop'}, inplace=True)
dat.rename(columns={'target_end_date': 'wk_end_date'}, inplace=True)
dat = dat[['wk_end_date', 'location', 'inc', 'pop']]
dat = dat[dat['wk_end_date'] < pd.Timestamp(ref_date)]
dat

,wk_end_date,location,inc,pop
0,2016-01-02,Bronx,12.163872,1356476
1,2016-01-02,Brooklyn,6.481274,2561225
2,2016-01-02,Manhattan,5.258377,1597451
3,2016-01-02,NYC,9.009407,8258035
4,2016-01-02,Queens,12.432311,2252196
...,...,...,...,...
2863,2025-02-22,Brooklyn,41.035052,2561225
2864,2025-02-22,Manhattan,30.423468,1597451
2865,2025-02-22,NYC,43.848204,8258035
2866,2025-02-22,Queens,42.358658,2252196


In [31]:
dat['wk_end_date'].max()

Timestamp('2025-02-22 00:00:00')

In [32]:
# Create epiweek and year columns
dat_arrange = dat.assign(
    epiweek=dat['wk_end_date'].dt.isocalendar().week,
    year=dat['wk_end_date'].dt.year
)


dat_arrange['season_week'] = loader.convert_epiweek_to_season_week(dat_arrange['year'].to_numpy(), dat_arrange['epiweek'].to_numpy())
dat_arrange = loader.adjust_year_based_on_target_end_date(dat_arrange)
dat_arrange = dat_arrange.assign(
    season=lambda x: (
        x['year'] - ((x['epiweek'] <= 30) & (x['season_week'] >= 1))
    ).astype(str) + "/" +
    (
        (x['year'] - ((x['epiweek'] <= 30) & (x['season_week'] >= 1)) + 1)
    ).astype(str).str[-2:]
)

df = dat_arrange
df


,wk_end_date,location,inc,pop,epiweek,year,season_week,season
0,2016-01-02,Bronx,12.163872,1356476,53,2015,23,2015/16
1,2016-01-02,Brooklyn,6.481274,2561225,53,2015,23,2015/16
2,2016-01-02,Manhattan,5.258377,1597451,53,2015,23,2015/16
3,2016-01-02,NYC,9.009407,8258035,53,2015,23,2015/16
4,2016-01-02,Queens,12.432311,2252196,53,2015,23,2015/16
...,...,...,...,...,...,...,...,...
2863,2025-02-22,Brooklyn,41.035052,2561225,8,2025,31,2024/25
2864,2025-02-22,Manhattan,30.423468,1597451,8,2025,31,2024/25
2865,2025-02-22,NYC,43.848204,8258035,8,2025,31,2024/25
2866,2025-02-22,Queens,42.358658,2252196,8,2025,31,2024/25


In [33]:
df[(df['wk_end_date'] == '2025-01-04') ]

,wk_end_date,location,inc,pop,epiweek,year,season_week,season
2820,2025-01-04,Bronx,102.176522,1356476,1,2025,24,2024/25
2821,2025-01-04,Brooklyn,64.695605,2561225,1,2025,24,2024/25
2822,2025-01-04,Manhattan,52.708972,1597451,1,2025,24,2024/25
2823,2025-01-04,NYC,70.949082,8258035,1,2025,24,2024/25
2824,2025-01-04,Queens,70.020549,2252196,1,2025,24,2024/25
2825,2025-01-04,Staten Island,39.740201,490687,1,2025,24,2024/25


In [34]:
df[(df['year'] == 2020) & (df['epiweek'] == 53)]

,wk_end_date,location,inc,pop,epiweek,year,season_week,season
1566,2021-01-02,Bronx,12.163872,1356476,53,2020,23,2020/21
1567,2021-01-02,Brooklyn,7.418325,2561225,53,2020,23,2020/21
1568,2021-01-02,Manhattan,5.884375,1597451,53,2020,23,2020/21
1569,2021-01-02,NYC,8.464483,8258035,53,2020,23,2020/21
1570,2021-01-02,Queens,9.146628,2252196,53,2020,23,2020/21
1571,2021-01-02,Staten Island,3.668326,490687,53,2020,23,2020/21


In [35]:
df.wk_end_date.max()

Timestamp('2025-02-22 00:00:00')

In [36]:
df['log_pop'] = np.log(df['pop'])
df['inc_4rt'] = (df['inc'] + 0.01)**0.25
df['inc_4rt_scale_factor'] = df \
.assign(inc_4rt_in_season = lambda x: np.where((x['season_week'] < 10) | (x['season_week'] > 45),
                                                np.nan,
                                                x['inc_4rt'])) \
.groupby(['location'])['inc_4rt_in_season'] \
.transform(lambda x: x.quantile(0.95))

df['inc_4rt_cs'] = df['inc_4rt'] / (df['inc_4rt_scale_factor'] + 0.01)
df['inc_4rt_center_factor'] = df \
.assign(inc_4rt_cs_in_season = lambda x: np.where((x['season_week'] < 10) | (x['season_week'] > 45),
                                                np.nan,
                                                x['inc_4rt_cs'])) \
.groupby(['location'])['inc_4rt_cs_in_season'] \
.transform(lambda x: x.mean())
df['inc_4rt_cs'] = df['inc_4rt_cs'] - df['inc_4rt_center_factor']
df
  

,wk_end_date,location,inc,pop,epiweek,year,season_week,season,log_pop,inc_4rt,inc_4rt_scale_factor,inc_4rt_cs,inc_4rt_center_factor
0,2016-01-02,Bronx,12.163872,1356476,53,2015,23,2015/16,14.120401,1.867915,3.273333,-0.185002,0.753911
1,2016-01-02,Brooklyn,6.481274,2561225,53,2015,23,2015/16,14.755996,1.596182,2.831156,-0.195892,0.757699
2,2016-01-02,Manhattan,5.258377,1597451,53,2015,23,2015/16,14.283920,1.515023,2.719978,-0.202689,0.757647
3,2016-01-02,NYC,9.009407,8258035,53,2015,23,2015/16,15.926697,1.732984,2.941963,-0.172601,0.759662
4,2016-01-02,Queens,12.432311,2252196,53,2015,23,2015/16,14.627416,1.878128,2.954197,-0.132167,0.765772
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,2025-02-22,Brooklyn,41.035052,2561225,8,2025,31,2024/25,14.755996,2.531134,2.831156,0.133183,0.757699
2864,2025-02-22,Manhattan,30.423468,1597451,8,2025,31,2024/25,14.283920,2.348756,2.719978,0.102710,0.757647
2865,2025-02-22,NYC,43.848204,8258035,8,2025,31,2024/25,15.926697,2.573432,2.941963,0.112107,0.759662
2866,2025-02-22,Queens,42.358658,2252196,8,2025,31,2024/25,14.627416,2.551298,2.954197,0.094933,0.765772


/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/2852538087.py:18: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/2852538087.py:18: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/2852538087.py:18: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/2852538087.py:18: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/2852538087.py:18: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.lineplot(
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm

In [37]:
# Filter the DataFrame based on multiple conditions
# Exclude data from the 2020/21 and 2021/22 flu seasons (likely due to COVID-19 disruptions)
# Exclude data between March 1, 2020, and September 1, 2020 (likely due to COVID-19 disruptions)   
filtered_df = df[
    ~df['season'].isin(["2020/21", "2021/22"]) &
    ~((df['wk_end_date'] >= datetime(2020, 3, 1)) & (df['wk_end_date'] <= datetime(2020, 9, 1)))
]
# location 
locations = filtered_df['location'].unique()
nhsn_to_plot = filtered_df.assign(season_loc = lambda x: x['season'] + '_' + x['location'])

fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharex=True, sharey=True)
axes = axes.flatten()  # 2D 배열을 1D로 평탄화

# figures by locations
for i, location in enumerate(locations):
    ax = axes[i]  # 각 subplot에 접근
    sns.lineplot(
        data=nhsn_to_plot[nhsn_to_plot['location'] == location],  # 특정 location 데이터만 선택
        x='season_week',
        y='inc_4rt_cs',
        units='season_loc',
        hue='season_loc',  # 색상은 log_pop 기준
        estimator=None,
        ci=None,
        ax=ax
    )
    ax.set_title(location)  # subplot 제목 설정
    ax.legend(title="Log Pop", fontsize=8)  # 범례 설정

# 빈 subplot 숨기기 (location 수가 6보다 적을 경우)
for j in range(len(locations), len(axes)):
    fig.delaxes(axes[j])

fig.tight_layout()
plt.show()

In [38]:
df = filtered_df

# we already have incidence (transformed scale), season week and log population
feat_names = ['inc_4rt_cs', 'season_week', 'log_pop']

# one-hot encodings of data source, agg_level, and location
for c in ['location']:
    ohe = pd.get_dummies(df[c], prefix=c)
    df = pd.concat([df, ohe], axis=1)
    feat_names = feat_names + list(ohe.columns)

# season week relative to christmas
df = df.merge(
    loader.get_holidays() \
      .query("holiday == 'Christmas Day'") \
      .drop(columns=['holiday', 'date']) \
      .rename(columns={'season_week': 'xmas_week'}),
    how='left',
    on='season') \
  .assign(delta_xmas = lambda x: x['season_week'] - x['xmas_week'])

feat_names = feat_names + ['delta_xmas']

In [39]:
df

,wk_end_date,location,inc,pop,epiweek,year,season_week,season,log_pop,inc_4rt,...,inc_4rt_cs,inc_4rt_center_factor,location_Bronx,location_Brooklyn,location_Manhattan,location_NYC,location_Queens,location_Staten Island,xmas_week,delta_xmas
0,2016-01-02,Bronx,12.163872,1356476,53,2015,23,2015/16,14.120401,1.867915,...,-0.185002,0.753911,1,0,0,0,0,0,21.0,2.0
1,2016-01-02,Brooklyn,6.481274,2561225,53,2015,23,2015/16,14.755996,1.596182,...,-0.195892,0.757699,0,1,0,0,0,0,21.0,2.0
2,2016-01-02,Manhattan,5.258377,1597451,53,2015,23,2015/16,14.283920,1.515023,...,-0.202689,0.757647,0,0,1,0,0,0,21.0,2.0
3,2016-01-02,NYC,9.009407,8258035,53,2015,23,2015/16,15.926697,1.732984,...,-0.172601,0.759662,0,0,0,1,0,0,21.0,2.0
4,2016-01-02,Queens,12.432311,2252196,53,2015,23,2015/16,14.627416,1.878128,...,-0.132167,0.765772,0,0,0,0,1,0,21.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2025-02-22,Brooklyn,41.035052,2561225,8,2025,31,2024/25,14.755996,2.531134,...,0.133183,0.757699,0,1,0,0,0,0,NaN,NaN
2108,2025-02-22,Manhattan,30.423468,1597451,8,2025,31,2024/25,14.283920,2.348756,...,0.102710,0.757647,0,0,1,0,0,0,NaN,NaN
2109,2025-02-22,NYC,43.848204,8258035,8,2025,31,2024/25,15.926697,2.573432,...,0.112107,0.759662,0,0,0,1,0,0,NaN,NaN
2110,2025-02-22,Queens,42.358658,2252196,8,2025,31,2024/25,14.627416,2.551298,...,0.094933,0.765772,0,0,0,0,1,0,NaN,NaN


In [40]:
feat_names

['inc_4rt_cs',
 'season_week',
 'log_pop',
 'location_Bronx',
 'location_Brooklyn',
 'location_Manhattan',
 'location_NYC',
 'location_Queens',
 'location_Staten Island',
 'delta_xmas']

In [41]:
df, new_feat_names = featurize.featurize_data(
    df, group_columns=['location'],
    features = [
        {
            'fun': 'windowed_taylor_coefs',
            'args': {
                'columns': 'inc_4rt_cs',
                'taylor_degree': 2,
                'window_align': 'trailing',
                'window_size': [4, 6],
                'fill_edges': False
            }
        },
        {
            'fun': 'windowed_taylor_coefs',
            'args': {
                'columns': 'inc_4rt_cs',
                'taylor_degree': 1,
                'window_align': 'trailing',
                'window_size': [3, 5],
                'fill_edges': False
            }
        },
        {
            'fun': 'rollmean',
            'args': {
                'columns': 'inc_4rt_cs',
                'group_columns': ['location'],
                'window_size': [2, 4]
            }
        }
    ])
feat_names = feat_names + new_feat_names

df, new_feat_names = featurize.featurize_data(
    df, group_columns=['location'],
    features = [
        {
            'fun': 'lag',
            'args': {
                'columns': ['inc_4rt_cs'] + new_feat_names,
                'lags': [1, 2]
            }
        }
    ])
feat_names = feat_names + new_feat_names

print(df)
print(feat_names)

     wk_end_date       location        inc      pop  epiweek  year  \
0     2016-01-02          Bronx  12.163872  1356476       53  2015   
1     2016-01-02       Brooklyn   6.481274  2561225       53  2015   
2     2016-01-02      Manhattan   5.258377  1597451       53  2015   
3     2016-01-02            NYC   9.009407  8258035       53  2015   
4     2016-01-02         Queens  12.432311  2252196       53  2015   
...          ...            ...        ...      ...      ...   ...   
2107  2025-02-22       Brooklyn  41.035052  2561225        8  2025   
2108  2025-02-22      Manhattan  30.423468  1597451        8  2025   
2109  2025-02-22            NYC  43.848204  8258035        8  2025   
2110  2025-02-22         Queens  42.358658  2252196        8  2025   
2111  2025-02-22  Staten Island  23.640325   490687        8  2025   

     season_week   season    log_pop   inc_4rt  ...  \
0             23  2015/16  14.120401  1.867915  ...   
1             23  2015/16  14.755996  1.596182  .

/Users/dk29776/Dropbox/UTAustin/Forecasting/flusion/src/timeseriesutils/src/timeseriesutils/featurize.py:507: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(taylor_coefs_one_column_grp,
/Users/dk29776/Dropbox/UTAustin/Forecasting/flusion/src/timeseriesutils/src/timeseriesutils/featurize.py:507: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this w

In [42]:
df.shape

(2112, 59)

In [43]:
new_feat_names

['inc_4rt_cs_lag1',
 'inc_4rt_cs_lag2',
 'inc_4rt_cs_taylor_d2_c0_w4t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c0_w4t_sNone_lag2',
 'inc_4rt_cs_taylor_d2_c1_w4t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c1_w4t_sNone_lag2',
 'inc_4rt_cs_taylor_d2_c2_w4t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c2_w4t_sNone_lag2',
 'inc_4rt_cs_taylor_d2_c0_w6t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c0_w6t_sNone_lag2',
 'inc_4rt_cs_taylor_d2_c1_w6t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c1_w6t_sNone_lag2',
 'inc_4rt_cs_taylor_d2_c2_w6t_sNone_lag1',
 'inc_4rt_cs_taylor_d2_c2_w6t_sNone_lag2',
 'inc_4rt_cs_taylor_d1_c0_w3t_sNone_lag1',
 'inc_4rt_cs_taylor_d1_c0_w3t_sNone_lag2',
 'inc_4rt_cs_taylor_d1_c1_w3t_sNone_lag1',
 'inc_4rt_cs_taylor_d1_c1_w3t_sNone_lag2',
 'inc_4rt_cs_taylor_d1_c0_w5t_sNone_lag1',
 'inc_4rt_cs_taylor_d1_c0_w5t_sNone_lag2',
 'inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag1',
 'inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag2',
 'inc_4rt_cs_rollmean_w2_lag1',
 'inc_4rt_cs_rollmean_w2_lag2',
 'inc_4rt_cs_rollmean_w4_lag1',
 'in

In [44]:
# import importlib
# featurize = importlib.reload(featurize)

# last date with observed data
# last_obs_date = pd.to_datetime(df.groupby(['location', 'pathogen_covid'])['date'].max().min())
# print(f'last_obs_date: {last_obs_date}')

# how far out to forecast to get to 28 days after due date
# due_date = forecast_date
# extra_horizons_rel_obs = (due_date - last_obs_date.date()).days
# effective_horizon_rel_obs = max_horizon + extra_horizons_rel_obs

# add forecast targets
df_with_pred_targets, new_feat_names = featurize.featurize_data(
    df, group_columns=['location'],
    features = [
        {
            'fun': 'horizon_targets',
            'args': {
                'columns': 'inc_4rt_cs',
                'horizons': [(i+1) for i in range(max_horizon)]
            }
        }
    ])
feat_names = feat_names + new_feat_names

# we will model the differences between the prediction target and the most
# recent observed value
df_with_pred_targets['delta_target'] = df_with_pred_targets['inc_4rt_cs_target'] - df_with_pred_targets['inc_4rt_cs']

print(new_feat_names)
print(df_with_pred_targets)

['horizon']
     wk_end_date       location        inc      pop  epiweek  year  \
0     2016-01-02          Bronx  12.163872  1356476       53  2015   
1     2016-01-02       Brooklyn   6.481274  2561225       53  2015   
2     2016-01-02      Manhattan   5.258377  1597451       53  2015   
3     2016-01-02            NYC   9.009407  8258035       53  2015   
4     2016-01-02         Queens  12.432311  2252196       53  2015   
...          ...            ...        ...      ...      ...   ...   
2107  2025-02-22       Brooklyn  41.035052  2561225        8  2025   
2108  2025-02-22      Manhattan  30.423468  1597451        8  2025   
2109  2025-02-22            NYC  43.848204  8258035        8  2025   
2110  2025-02-22         Queens  42.358658  2252196        8  2025   
2111  2025-02-22  Staten Island  23.640325   490687        8  2025   

     season_week   season    log_pop   inc_4rt  ...  \
0             23  2015/16  14.120401  1.867915  ...   
1             23  2015/16  14.755996 

In [45]:
df_with_pred_targets.shape

(10560, 62)

In [46]:
df_with_pred_targets.wk_end_date.max()

Timestamp('2025-02-22 00:00:00')

In [47]:
df_with_pred_targets[df_with_pred_targets['inc_4rt_cs_target'].isna()]


,wk_end_date,location,inc,pop,epiweek,year,season_week,season,log_pop,inc_4rt,...,inc_4rt_cs_taylor_d1_c0_w5t_sNone_lag2,inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag1,inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag2,inc_4rt_cs_rollmean_w2_lag1,inc_4rt_cs_rollmean_w2_lag2,inc_4rt_cs_rollmean_w4_lag1,inc_4rt_cs_rollmean_w4_lag2,inc_4rt_cs_target,horizon,delta_target
2106,2025-02-22,Bronx,65.758628,1356476,8,2025,31,2024/25,14.120401,2.847769,...,0.239293,-0.025349,-0.000678,0.051671,-0.033154,0.034914,-0.050355,NaN,1,NaN
2107,2025-02-22,Brooklyn,41.035052,2561225,8,2025,31,2024/25,14.755996,2.531134,...,0.245006,-0.028930,-0.006091,0.092268,-0.051212,0.062059,-0.044170,NaN,1,NaN
2108,2025-02-22,Manhattan,30.423468,1597451,8,2025,31,2024/25,14.283920,2.348756,...,0.228786,-0.024343,-0.001932,0.115553,-0.047371,0.083612,-0.040262,NaN,1,NaN
2109,2025-02-22,NYC,43.848204,8258035,8,2025,31,2024/25,15.926697,2.573432,...,0.221868,-0.025978,-0.004957,0.190112,-0.024389,0.141190,-0.037800,NaN,1,NaN
2110,2025-02-22,Queens,42.358658,2252196,8,2025,31,2024/25,14.627416,2.551298,...,0.187413,-0.024556,-0.009231,0.255680,0.018157,0.185617,-0.014607,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2025-02-22,Brooklyn,41.035052,2561225,8,2025,31,2024/25,14.755996,2.531134,...,0.245006,-0.028930,-0.006091,0.092268,-0.051212,0.062059,-0.044170,NaN,5,NaN
2108,2025-02-22,Manhattan,30.423468,1597451,8,2025,31,2024/25,14.283920,2.348756,...,0.228786,-0.024343,-0.001932,0.115553,-0.047371,0.083612,-0.040262,NaN,5,NaN
2109,2025-02-22,NYC,43.848204,8258035,8,2025,31,2024/25,15.926697,2.573432,...,0.221868,-0.025978,-0.004957,0.190112,-0.024389,0.141190,-0.037800,NaN,5,NaN
2110,2025-02-22,Queens,42.358658,2252196,8,2025,31,2024/25,14.627416,2.551298,...,0.187413,-0.024556,-0.009231,0.255680,0.018157,0.185617,-0.014607,NaN,5,NaN


In [48]:
df_with_pred_targets2 = df_with_pred_targets.query("season_week >= 5 and season_week <= 45")

In [49]:
df_with_pred_targets2.shape

(8400, 62)

In [50]:
#date_list <- c("2023-10-01", "2023-11-05", "2023-12-10", "2024-01-14", "2024-02-18", "2024-03-24")

In [51]:
#df_with_pred_targets = df_with_pred_targets2
df_with_pred_targets = df_with_pred_targets2.drop(columns=['epiweek'])
    

# "test set" df used to generate look-ahead predictions
df_test = df_with_pred_targets \
    .loc[df_with_pred_targets.wk_end_date == df_with_pred_targets.wk_end_date.max()] \
    .copy()

x_test = df_test[feat_names]

# "train set" df for model fitting; target value non-missing
df_train = df_with_pred_targets.loc[~df_with_pred_targets['delta_target'].isna().values]
x_train = df_train[feat_names]
y_train = df_train['delta_target']

In [52]:
df_test

,wk_end_date,location,inc,pop,year,season_week,season,log_pop,inc_4rt,inc_4rt_scale_factor,...,inc_4rt_cs_taylor_d1_c0_w5t_sNone_lag2,inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag1,inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag2,inc_4rt_cs_rollmean_w2_lag1,inc_4rt_cs_rollmean_w2_lag2,inc_4rt_cs_rollmean_w4_lag1,inc_4rt_cs_rollmean_w4_lag2,inc_4rt_cs_target,horizon,delta_target
2106,2025-02-22,Bronx,65.758628,1356476,2025,31,2024/25,14.120401,2.847769,3.273333,...,0.239293,-0.025349,-0.000678,0.051671,-0.033154,0.034914,-0.050355,NaN,1,NaN
2107,2025-02-22,Brooklyn,41.035052,2561225,2025,31,2024/25,14.755996,2.531134,2.831156,...,0.245006,-0.028930,-0.006091,0.092268,-0.051212,0.062059,-0.044170,NaN,1,NaN
2108,2025-02-22,Manhattan,30.423468,1597451,2025,31,2024/25,14.283920,2.348756,2.719978,...,0.228786,-0.024343,-0.001932,0.115553,-0.047371,0.083612,-0.040262,NaN,1,NaN
2109,2025-02-22,NYC,43.848204,8258035,2025,31,2024/25,15.926697,2.573432,2.941963,...,0.221868,-0.025978,-0.004957,0.190112,-0.024389,0.141190,-0.037800,NaN,1,NaN
2110,2025-02-22,Queens,42.358658,2252196,2025,31,2024/25,14.627416,2.551298,2.954197,...,0.187413,-0.024556,-0.009231,0.255680,0.018157,0.185617,-0.014607,NaN,1,NaN
2111,2025-02-22,Staten Island,23.640325,490687,2025,31,2024/25,13.103562,2.205257,2.505093,...,0.193313,-0.029436,-0.008172,0.228779,0.031849,0.209445,0.003730,NaN,1,NaN
2106,2025-02-22,Bronx,65.758628,1356476,2025,31,2024/25,14.120401,2.847769,3.273333,...,0.239293,-0.025349,-0.000678,0.051671,-0.033154,0.034914,-0.050355,NaN,2,NaN
2107,2025-02-22,Brooklyn,41.035052,2561225,2025,31,2024/25,14.755996,2.531134,2.831156,...,0.245006,-0.028930,-0.006091,0.092268,-0.051212,0.062059,-0.044170,NaN,2,NaN
2108,2025-02-22,Manhattan,30.423468,1597451,2025,31,2024/25,14.283920,2.348756,2.719978,...,0.228786,-0.024343,-0.001932,0.115553,-0.047371,0.083612,-0.040262,NaN,2,NaN
2109,2025-02-22,NYC,43.848204,8258035,2025,31,2024/25,15.926697,2.573432,2.941963,...,0.221868,-0.025978,-0.004957,0.190112,-0.024389,0.141190,-0.037800,NaN,2,NaN


In [53]:
import warnings


# seed for random number generation, based on reference date
rng_seed = int(time.mktime(ref_date.timetuple()))
rng = np.random.default_rng(seed=rng_seed)
# seeds for lgb model fits, one per combination of bag and quantile level
lgb_seeds = rng.integers(1e8, size=(num_bags, len(q_levels)))

# training loop over bags
# for q_level, q_label in zip(q_levels, q_labels):
    # print(q_label)

oob_preds_by_bag = np.empty((x_train.shape[0], num_bags, len(q_levels)))
oob_preds_by_bag[:] = np.nan
test_preds_by_bag = np.empty((x_test.shape[0], num_bags, len(q_levels)))

train_seasons = df_train['season'].unique()

# Convert season_week and delta_xmas to numeric
x_train['season_week'] = pd.to_numeric(x_train['season_week'], errors='coerce')
x_train['delta_xmas'] = pd.to_numeric(x_train['delta_xmas'], errors='coerce')

x_test['season_week'] = pd.to_numeric(x_test['season_week'], errors='coerce')
x_test['delta_xmas'] = pd.to_numeric(x_test['delta_xmas'], errors='coerce')

#print(x_train.dtypes[x_train.dtypes == 'object'])
#print(x_test.dtypes[x_test.dtypes == 'object'])
# Initialize a DataFrame to store feature importances
feature_importance_df = pd.DataFrame(0, index=x_train.columns, columns=range(num_bags * len(q_levels)))

# Suppress specific warnings
warnings.simplefilter(action="ignore", category=DeprecationWarning)

for b in range(num_bags):
    print(f'bag number {b+1}')
    # get indices of observations that are in bag
    bag_seasons = rng.choice(
        train_seasons,
        size = int(len(train_seasons) * bag_frac_samples),
        replace=False)
    bag_obs_inds = df_train['season'].isin(bag_seasons)
    
    for q_ind, q_level in enumerate(q_levels):
        # fit to bag
        model = lgb.LGBMRegressor(verbosity=-1,
                                    # hist_pool_size=1000,
                                    # objective='mean_absolute_error')
                                    objective='quantile',
                                    alpha = q_level,
                                    random_state=lgb_seeds[b, q_ind])
        model.fit(X=x_train.loc[bag_obs_inds, :], y=y_train.loc[bag_obs_inds])
        
        # oob predictions and test set predictions
        #oob_preds_by_bag[~bag_obs_inds, b, q_ind] = model.predict(X=x_train.loc[~bag_obs_inds, :])
        test_preds_by_bag[:, b, q_ind] = model.predict(X=x_test)

        # Save feature importances
        col_idx = b * len(q_levels) + q_ind
        feature_importance_df.iloc[:, col_idx] = model.feature_importances_


# df_train['oob_pred'] = np.nanmedian(oob_preds_by_bag, axis=1)
# df_test['delta_hat_stage1'] = np.median(test_preds_by_bag, axis=1)


/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/417496418.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['season_week'] = pd.to_numeric(x_train['season_week'], errors='coerce')
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/417496418.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['delta_xmas'] = pd.to_numeric(x_train['delta_xmas'], errors='coerce')
/var/folders/57/pf1ss2_n1sn5cdzq_nk8v7sm0000gr/T/ipykernel_90767/417496418.py:24: SettingWithCopyW

bag number 1
bag number 2
bag number 3
bag number 4
bag number 5
bag number 6
bag number 7
bag number 8
bag number 9
bag number 10
bag number 11
bag number 12
bag number 13
bag number 14
bag number 15
bag number 16
bag number 17
bag number 18
bag number 19
bag number 20
bag number 21
bag number 22
bag number 23
bag number 24
bag number 25
bag number 26
bag number 27
bag number 28
bag number 29
bag number 30
bag number 31
bag number 32
bag number 33
bag number 34
bag number 35
bag number 36
bag number 37
bag number 38
bag number 39
bag number 40
bag number 41
bag number 42
bag number 43
bag number 44
bag number 45
bag number 46
bag number 47
bag number 48
bag number 49
bag number 50
bag number 51
bag number 52
bag number 53
bag number 54
bag number 55
bag number 56
bag number 57
bag number 58
bag number 59
bag number 60
bag number 61
bag number 62
bag number 63
bag number 64
bag number 65
bag number 66
bag number 67
bag number 68
bag number 69
bag number 70
bag number 71
bag number 72
b

In [54]:
feature_importance_df['average_importance'] = feature_importance_df.mean(axis=1)
feature_importance_df = feature_importance_df.sort_values(by='average_importance', ascending=False)
feature_importance_df

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,average_importance
horizon,268,305,337,295,317,318,296,283,214,254,...,241,311,355,342,356,344,304,294,175,299.324444
season_week,155,177,182,176,171,193,172,174,185,120,...,166,178,197,248,220,216,222,202,155,182.786667
inc_4rt_cs,168,79,80,79,96,87,87,126,153,161,...,167,94,75,69,80,99,79,100,179,106.534444
inc_4rt_cs_rollmean_w2,93,91,94,102,94,111,94,105,142,119,...,125,118,89,102,106,89,117,91,170,105.158889
delta_xmas,103,132,149,158,132,149,156,133,114,151,...,48,44,36,36,69,39,45,59,35,95.893333
inc_4rt_cs_rollmean_w4_lag2,92,71,90,115,118,119,109,101,123,80,...,102,94,89,90,94,86,90,68,53,94.832222
inc_4rt_cs_taylor_d1_c1_w5t_sNone_lag2,94,70,68,63,81,95,100,56,59,57,...,69,96,99,107,110,127,105,102,108,92.975556
inc_4rt_cs_rollmean_w4,105,139,119,112,102,107,91,96,81,124,...,83,100,87,82,85,114,81,81,85,90.245556
inc_4rt_cs_taylor_d1_c1_w5t_sNone,84,69,69,79,61,59,69,70,63,88,...,65,78,89,99,94,100,107,101,101,87.587778
inc_4rt_cs_taylor_d2_c1_w4t_sNone,92,80,82,82,74,77,72,100,117,100,...,123,107,96,77,65,88,89,79,104,87.030000


In [55]:
feature_importance_df['average_importance'].head(10).plot(kind='barh', figsize=(10, 6))
plt.title("Top 10 Most Important Features")
plt.xlabel("Feature Importance")
plt.ylabel("Feature")
plt.gca().invert_yaxis()
plt.show()

In [56]:
#oob_pred_qs = np.nanmedian(oob_preds_by_bag, axis=1)
test_pred_qs = np.median(test_preds_by_bag, axis=1)
test_pred_qs_sorted = np.sort(test_pred_qs, axis=1)  # Sort to enforce monotonicity
test_pred_qs_sorted.shape

test_pred_qs_df = pd.DataFrame(test_pred_qs_sorted)
test_pred_qs_df.columns = q_labels
test_pred_qs_df

,0.025,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.975
0,-0.088953,-0.071448,-0.062412,-0.039717,-0.036380,-0.026820,-0.019138,-0.009291,0.009080
1,-0.094337,-0.072457,-0.055908,-0.042986,-0.037995,-0.025073,-0.018678,-0.012270,0.009347
2,-0.082306,-0.064210,-0.052196,-0.037053,-0.028983,-0.022495,-0.012207,-0.003675,0.008577
3,-0.074994,-0.059225,-0.050668,-0.031410,-0.030627,-0.015785,-0.010626,-0.002148,0.013048
4,-0.075421,-0.056236,-0.042839,-0.021560,-0.020954,-0.014747,-0.005380,0.002604,0.011113
5,-0.080314,-0.065186,-0.053069,-0.034729,-0.032090,-0.021141,0.001461,0.009140,0.022289
6,-0.102553,-0.091701,-0.087200,-0.077438,-0.068317,-0.049096,-0.028157,-0.010391,0.011061
7,-0.110027,-0.089011,-0.073969,-0.071179,-0.067237,-0.045057,-0.028600,-0.014704,0.010501
8,-0.096801,-0.082986,-0.071668,-0.070418,-0.060070,-0.046500,-0.024390,-0.006421,0.009809
9,-0.084433,-0.077425,-0.068557,-0.051430,-0.050565,-0.031630,-0.017079,-0.002639,0.013007


In [57]:
df_test.reset_index(drop=True, inplace=True)
df_test_w_preds = pd.concat([df_test, test_pred_qs_df], axis=1)
df_test_w_preds

,wk_end_date,location,inc,pop,year,season_week,season,log_pop,inc_4rt,inc_4rt_scale_factor,...,delta_target,0.025,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.975
0,2025-02-22,Bronx,65.758628,1356476,2025,31,2024/25,14.120401,2.847769,3.273333,...,NaN,-0.088953,-0.071448,-0.062412,-0.039717,-0.036380,-0.026820,-0.019138,-0.009291,0.009080
1,2025-02-22,Brooklyn,41.035052,2561225,2025,31,2024/25,14.755996,2.531134,2.831156,...,NaN,-0.094337,-0.072457,-0.055908,-0.042986,-0.037995,-0.025073,-0.018678,-0.012270,0.009347
2,2025-02-22,Manhattan,30.423468,1597451,2025,31,2024/25,14.283920,2.348756,2.719978,...,NaN,-0.082306,-0.064210,-0.052196,-0.037053,-0.028983,-0.022495,-0.012207,-0.003675,0.008577
3,2025-02-22,NYC,43.848204,8258035,2025,31,2024/25,15.926697,2.573432,2.941963,...,NaN,-0.074994,-0.059225,-0.050668,-0.031410,-0.030627,-0.015785,-0.010626,-0.002148,0.013048
4,2025-02-22,Queens,42.358658,2252196,2025,31,2024/25,14.627416,2.551298,2.954197,...,NaN,-0.075421,-0.056236,-0.042839,-0.021560,-0.020954,-0.014747,-0.005380,0.002604,0.011113
5,2025-02-22,Staten Island,23.640325,490687,2025,31,2024/25,13.103562,2.205257,2.505093,...,NaN,-0.080314,-0.065186,-0.053069,-0.034729,-0.032090,-0.021141,0.001461,0.009140,0.022289
6,2025-02-22,Bronx,65.758628,1356476,2025,31,2024/25,14.120401,2.847769,3.273333,...,NaN,-0.102553,-0.091701,-0.087200,-0.077438,-0.068317,-0.049096,-0.028157,-0.010391,0.011061
7,2025-02-22,Brooklyn,41.035052,2561225,2025,31,2024/25,14.755996,2.531134,2.831156,...,NaN,-0.110027,-0.089011,-0.073969,-0.071179,-0.067237,-0.045057,-0.028600,-0.014704,0.010501
8,2025-02-22,Manhattan,30.423468,1597451,2025,31,2024/25,14.283920,2.348756,2.719978,...,NaN,-0.096801,-0.082986,-0.071668,-0.070418,-0.060070,-0.046500,-0.024390,-0.006421,0.009809
9,2025-02-22,NYC,43.848204,8258035,2025,31,2024/25,15.926697,2.573432,2.941963,...,NaN,-0.084433,-0.077425,-0.068557,-0.051430,-0.050565,-0.031630,-0.017079,-0.002639,0.013007


In [58]:
# delta_names = [f'delta_hat_q{q_label}' for q_label in q_labels]
cols_to_keep = ['wk_end_date' ,'location', 'pop',
                'inc_4rt_cs', 'horizon',
                'inc_4rt_center_factor', 'inc_4rt_scale_factor']
preds_df = df_test_w_preds[cols_to_keep + q_labels]
preds_df = pd.melt(preds_df,
                id_vars=cols_to_keep,
                var_name='quantile',
                value_name = 'delta_hat')
preds_df

,wk_end_date,location,pop,inc_4rt_cs,horizon,inc_4rt_center_factor,inc_4rt_scale_factor,quantile,delta_hat
0,2025-02-22,Bronx,1356476,0.113430,1,0.753911,3.273333,0.025,-0.088953
1,2025-02-22,Brooklyn,2561225,0.133183,1,0.757699,2.831156,0.025,-0.094337
2,2025-02-22,Manhattan,1597451,0.102710,1,0.757647,2.719978,0.025,-0.082306
3,2025-02-22,NYC,8258035,0.112107,1,0.759662,2.941963,0.025,-0.074994
4,2025-02-22,Queens,2252196,0.094933,1,0.765772,2.954197,0.025,-0.075421
...,...,...,...,...,...,...,...,...,...
265,2025-02-22,Brooklyn,2561225,0.133183,5,0.757699,2.831156,0.975,0.010742
266,2025-02-22,Manhattan,1597451,0.102710,5,0.757647,2.719978,0.975,0.007716
267,2025-02-22,NYC,8258035,0.112107,5,0.759662,2.941963,0.975,0.012928
268,2025-02-22,Queens,2252196,0.094933,5,0.765772,2.954197,0.975,0.011036


In [59]:
# build data frame with predictions on the original scale
preds_df['inc_4rt_cs_target_hat'] = preds_df['inc_4rt_cs'] + preds_df['delta_hat']
preds_df['inc_4rt_target_hat'] = (preds_df['inc_4rt_cs_target_hat'] + preds_df['inc_4rt_center_factor']) * (preds_df['inc_4rt_scale_factor'] + 0.01)
preds_df['value'] = (np.maximum(preds_df['inc_4rt_target_hat'], 0.0) ** 4 - 0.01) * preds_df['pop'] / 100000
preds_df['value'] = np.maximum(preds_df['value'], 0.0)
preds_df

,wk_end_date,location,pop,inc_4rt_cs,horizon,inc_4rt_center_factor,inc_4rt_scale_factor,quantile,delta_hat,inc_4rt_cs_target_hat,inc_4rt_target_hat,value
0,2025-02-22,Bronx,1356476,0.113430,1,0.753911,3.273333,0.025,-0.088953,0.024478,2.555707,578.568512
1,2025-02-22,Brooklyn,2561225,0.133183,1,0.757699,2.831156,0.025,-0.094337,0.038846,2.263109,671.589188
2,2025-02-22,Manhattan,1597451,0.102710,1,0.757647,2.719978,0.025,-0.082306,0.020404,2.124063,325.000209
3,2025-02-22,NYC,8258035,0.112107,1,0.759662,2.941963,0.025,-0.074994,0.037114,2.352053,2526.522157
4,2025-02-22,Queens,2252196,0.094933,1,0.765772,2.954197,0.025,-0.075421,0.019512,2.327736,660.987870
...,...,...,...,...,...,...,...,...,...,...,...,...
265,2025-02-22,Brooklyn,2561225,0.133183,5,0.757699,2.831156,0.975,0.010742,0.143925,2.561654,1102.626838
266,2025-02-22,Manhattan,1597451,0.102710,5,0.757647,2.719978,0.975,0.007716,0.110426,2.369821,503.676523
267,2025-02-22,NYC,8258035,0.112107,5,0.759662,2.941963,0.975,0.012928,0.125035,2.611594,3840.661040
268,2025-02-22,Queens,2252196,0.094933,5,0.765772,2.954197,0.975,0.011036,0.105969,2.584012,1003.892050


In [60]:
# keep just required columns and rename to match hub format
preds_df = preds_df[['wk_end_date', 'location', 'horizon', 'quantile', 'value']] \
    .rename(
        columns={
            'quantile': 'output_type_id'
        })

preds_df['target_end_date'] = preds_df['wk_end_date'] + pd.to_timedelta(7*preds_df['horizon'], unit='days')
preds_df['reference_date'] = ref_date
preds_df['target'] = 'ILI ED visits'
preds_df['horizon'] = preds_df['horizon'] - 1

preds_df['output_type'] = 'quantile'
preds_df



,wk_end_date,location,horizon,output_type_id,value,target_end_date,reference_date,target,output_type
0,2025-02-22,Bronx,0,0.025,578.568512,2025-03-01,2025-03-01,ILI ED visits,quantile
1,2025-02-22,Brooklyn,0,0.025,671.589188,2025-03-01,2025-03-01,ILI ED visits,quantile
2,2025-02-22,Manhattan,0,0.025,325.000209,2025-03-01,2025-03-01,ILI ED visits,quantile
3,2025-02-22,NYC,0,0.025,2526.522157,2025-03-01,2025-03-01,ILI ED visits,quantile
4,2025-02-22,Queens,0,0.025,660.987870,2025-03-01,2025-03-01,ILI ED visits,quantile
...,...,...,...,...,...,...,...,...,...
265,2025-02-22,Brooklyn,4,0.975,1102.626838,2025-03-29,2025-03-01,ILI ED visits,quantile
266,2025-02-22,Manhattan,4,0.975,503.676523,2025-03-29,2025-03-01,ILI ED visits,quantile
267,2025-02-22,NYC,4,0.975,3840.661040,2025-03-29,2025-03-01,ILI ED visits,quantile
268,2025-02-22,Queens,4,0.975,1003.892050,2025-03-29,2025-03-01,ILI ED visits,quantile


In [61]:
preds_df = preds_df[['reference_date', 'location', 'horizon', 'target', 'target_end_date', 'output_type', 'output_type_id', 'value']]
preds_df

,reference_date,location,horizon,target,target_end_date,output_type,output_type_id,value
0,2025-03-01,Bronx,0,ILI ED visits,2025-03-01,quantile,0.025,578.568512
1,2025-03-01,Brooklyn,0,ILI ED visits,2025-03-01,quantile,0.025,671.589188
2,2025-03-01,Manhattan,0,ILI ED visits,2025-03-01,quantile,0.025,325.000209
3,2025-03-01,NYC,0,ILI ED visits,2025-03-01,quantile,0.025,2526.522157
4,2025-03-01,Queens,0,ILI ED visits,2025-03-01,quantile,0.025,660.987870
...,...,...,...,...,...,...,...,...
265,2025-03-01,Brooklyn,4,ILI ED visits,2025-03-29,quantile,0.975,1102.626838
266,2025-03-01,Manhattan,4,ILI ED visits,2025-03-29,quantile,0.975,503.676523
267,2025-03-01,NYC,4,ILI ED visits,2025-03-29,quantile,0.975,3840.661040
268,2025-03-01,Queens,4,ILI ED visits,2025-03-29,quantile,0.975,1003.892050


In [62]:
if not Path('epiENGAGE-GBQR/model_output/NYC_ED').exists():
    Path('epiENGAGE-GBQR/model_output/NYC_ED').mkdir(parents=True)

preds_df.to_csv(f'epiENGAGE-GBQR/model_output/NYC_ED/{str(ref_date)}-epiENGAGE-GBQR.csv', index=False)
